# 1 dimensional laws

In [8]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import tensorflow as tf
from sympy import poly
from sympy.abc import x,y,z, k
from sympy import degree
from sympy import *
import sympy
import csv

# CSV Write or read works like this

see https://www.pythontutorial.net/python-basics/python-write-csv-file/

In [9]:
import csv

###Write###
# open the file in the write mode
f = open('path/to/csv_file', 'w')

# create the csv writer
writer = csv.writer(f)

# write a row to the csv file
writer.writerow(row)

# close the file
f.close()

###Read###

# open the file in the write mode
with open('path/to/csv_file', 'w') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    writer.writerow(row)


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/csv_file'

## Wolfram's 256 Rules

These are all cellular automaton rules for a 1-d array with only neares neighbour interaction. There are therefore only three cells (L,C,R) determining the outcome of one cell (C). This gives 2^3=8 possible combinations that can give rise to an outcome of 2 possibilities. Thus there are 2^8=256 possible rules. 

The following function generates each of these rules.
 
Input:  
num = number of rule  
inp = string of input bits   
t = number of timesteps calculated  

Output: an array of arrays each for one timestep beginning with the input array. 

The boudary conditions are periodic
        

In [10]:
def CellularAutomata(num,inp,t):
    bnum = np.flip(np.fromiter(np.binary_repr(num,width=8),dtype=int))
    out = [np.fromiter(inp,dtype=int)]
    lin = len(inp)
    currinp = inp
    for n in range(t):
        outnow = []
        for i in range(len(inp)):
            outnow = np.append(outnow,bnum[int(str(currinp[(i-1)%lin])+str(currinp[(i)%lin])+str(currinp[(i+1)%lin]),2)])

        currinp =''.join([str(int(elem)) for elem in outnow])
        out = np.append(out,[outnow],axis=0)
    return(out)


## Pol deg of rule

Resources on sympy for polynomials: https://docs.sympy.org/latest/modules/polys/reference.html  

Input: Rule number (integer)  
Output: Polynomial degree of rule (integer)

In [21]:
def one(a,x):
    u=1
    for i in range(len(x)):
        u=u*(1+(2*a[i]-1)*(2*x[i]-1))/2
    return(u.expand())

def ACAP(num):
    u=0
    x=[sympy.Symbol('x1'),sympy.Symbol('x2'),sympy.Symbol('x3')]
    for i in [0,1]:
        for j in [0,1]:
            for l in [0,1]:
                u=u+CellularAutomata(num,[i,j,l],1)[1,1]*one([i,j,l],x)
    return(u)
    
                
def PolyDegOfRule(num):
    return(sympy.Poly(ACAP(num)).total_degree())

## Generating Data

In [12]:
#N=10000 #batch size
#L=3 #Length of box
#rule= 22 #170 is movement to the left x_i' = x_{i+1}
#time = 1

def data_generation(N,L,rule,time):
    x_train=[]
    y_train=[]


    for i in range(N):
        #x=[0,0,0]
        #x[np.random.randint(0,3)]=1
        x=np.random.randint(0,2,L).astype(float)
        x_train.append(np.copy(x))
        #y_train.append(np.copy(CellularAutomata(rule,''.join([str(int(elem)) for elem in x]),time)[time]))
        y_train.append(np.copy(CellularAutomata(rule,''.join([str(int(elem)) for elem in x]),time)[time,1]))
        #print(''.join([str(int(elem)) for elem in x]),CellularAutomata(rule,''.join([str(int(elem)) for elem in x]),time)[time,0])

    #for i in range(20):
     #   print(x_train[i],y_train[i])

    x_train = np.array(x_train).reshape(N,L,1)
    y_train = np.array(y_train).reshape(N,1)
    return((x_train,y_train))
    #print(x_train.shape,y_train.shape)
    #print(x_train[0],y_train[0],CellularAutomata(rule,''.join([str(int(elem)) for elem in x_train[0]]),time)[time])

(x_train,y_train)=data_generation(10000,3,22,1)

## Models
useful resource for understanding what is going on:  
Padding https://machinelearningmastery.com/padding-and-stride-for-convolutional-neural-networks/  
Guide to 2d conv layers https://machinelearningmastery.com/padding-and-stride-for-convolutional-neural-networks/  
Guide to 1d conv layers https://machinelearningmastery.com/convolutional-layers-for-deep-learning-neural-networks/  
shapes, dimensions and units https://stackoverflow.com/questions/44747343/keras-input-explanation-input-shape-units-batch-size-dim-etc  


### Dense layers

### 1-D Convolutional layer

Tutorial custom layers: https://www.tensorflow.org/tutorials/customization/custom_layers

In [13]:
class SimpleConv(tf.keras.layers.Layer):
    def __init__(self, num_neighbours):
        super(SimpleConv, self).__init__()
        self.num_neighbours = num_neighbours

    def build(self, input_shape):
        self.kernel = self.add_weight("kernel",
                                      shape=[int(input_shape[-1]),
                                             self.num_neighbours])
#        self.kernel = self.add_weight("kernel",tf.constant([[0],[0],[1]]))
    #        self.bias = self.add_weight("bias",
#                                      shape=[int(input_shape[-1]),
 #                                            self.num_neighbours])

    def call(self, inputs):
        inshape = tf.shape(inputs)
#        return tf.matmul(inputs, self.kernel)
        return tf.reshape(tf.tensordot(inputs,self.kernel,axes=2),(inshape[0],1))
#        return tf.reshape(tf.tensordot(inputs,tf.constant([[0.,0,1]]),axes=2),(inshape[0],1))

layer = SimpleConv(3)
print(x_train[0:3])
print(y_train[0:3])
#layer.set_weights(np.array([1]))
print(layer(x_train[0:3]))
layer.set_weights([np.array([[0,0,1]])])
print(layer.get_weights())


[[[0.]
  [0.]
  [1.]]

 [[1.]
  [0.]
  [1.]]

 [[1.]
  [0.]
  [1.]]]
[[1.]
 [0.]
 [0.]]
tf.Tensor(
[[-1.1744114]
 [-1.3443956]
 [-1.3443956]], shape=(3, 1), dtype=float32)
[array([[0., 0., 1.]], dtype=float32)]


2022-05-06 16:18:45.929422: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-06 16:18:45.929444: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-06 16:18:45.929464: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (manu-ThinkPad-T460s): /proc/driver/nvidia/version does not exist
2022-05-06 16:18:45.929694: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
#layer = SimpleConv()
inp = x_train[0:5]

print(inp)
print( y_train[0:5])
#print(inp.shape)
#layer(inp)
#layer = tf.keras.layers.Conv1D(
#1,3, activation='relu', input_shape=((3,1)))
print(layer(inp))
print(layer.get_weights())


[[[0.]
  [0.]
  [1.]]

 [[1.]
  [0.]
  [1.]]

 [[1.]
  [0.]
  [1.]]

 [[1.]
  [1.]
  [1.]]

 [[0.]
  [1.]
  [0.]]]
[[1.]
 [0.]
 [0.]
 [0.]
 [1.]]
tf.Tensor(
[[1.]
 [1.]
 [1.]
 [1.]
 [0.]], shape=(5, 1), dtype=float32)
[array([[0., 0., 1.]], dtype=float32)]


In [18]:
def define_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer((3,1))) 
    #model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dense(3,activation="relu"))
    #model.add(tf.keras.layers.Reshape((3,1)))
    model.add(SimpleConv(3))
    model.add(tf.keras.layers.Reshape((1,1)))
    model.add(SimpleConv(1))
    model.add(tf.keras.layers.Reshape((1,1)))
    model.add(SimpleConv(1))
    model.add(tf.keras.layers.Reshape((1,1)))
    model.add(SimpleConv(1))
    model.add(tf.keras.layers.Reshape((1,1)))
    model.add(SimpleConv(1))
    model.add(tf.keras.layers.Reshape((1,1)))
    model.add(SimpleConv(1))
    model.add(tf.keras.layers.Reshape((1,1)))
    model.add(SimpleConv(1))
    model.add(tf.keras.layers.Reshape((1,1)))
    model.add(SimpleConv(1))
    model.add(tf.keras.layers.Reshape((1,1)))
    model.add(SimpleConv(1))
    model.add(tf.keras.layers.Reshape((1,1)))
    model.add(SimpleConv(1))
    model.add(tf.keras.layers.Reshape((1,1)))
    model.add(SimpleConv(1))
    model.add(tf.keras.layers.Reshape((1,1)))
    model.add(SimpleConv(1))
    #model.add(tf.keras.layers.Conv1D(1,3, activation='softmax', input_shape=((3,1))))
    #model.add(tf.keras.layers.Conv1D(100,3, activation='relu', input_shape=((1,100))))

    #model.add(tf.keras.layers.Softmax())
    #tf.keras.layers.Activation('sigmoid')

    #model.add(tf.keras.activations.sigmoid())
    #model.add(tf.keras.layers.Dense(3,activation="relu"))
    #model.add(tf.keras.layers.Dense(1,activation="softmax"))
    #model.layers[0].set_weights([np.array([[1,1,1]])])#Initializing with almost the right law converges, but never with sometheing a bit more off
    #model.summary()
    return(model)

## Execute learning

In [19]:
def learn(inp_model,x_train,y_train,epochs,verbose):
    #loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True) #Logits true means output values are any real numbers
    loss_fn = tf.keras.losses.MeanSquaredError()
    #loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    predictions = inp_model(x_train).numpy()
    #print(loss_fn(y_train, predictions).numpy())

    inp_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                  loss=loss_fn,
                  #metrics=[tf.keras.metrics.BinaryAccuracy()]
                  metrics=['accuracy']
                    #metrics= [tf.keras.losses.CategoricalCrossentropy()]
                 )
    train_history = model.fit(x=x_train, y=y_train, epochs=epochs,verbose=verbose)
  #  plt.title("Loss")
   # plt.plot(train_history.history['loss'], 'k')
    return(model,train_history)
model=define_model()
(x_train,y_train) = data_generation(1000,3,170,1)
learn(model,x_train,y_train,5,1)[1].history['accuracy'][-1]

2022-05-06 16:19:29.570461: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
32/32 [==============================] - 1s 2ms/step - loss: 0.1147 - accuracy: 0.8870
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 9.3183e-05 - accuracy: 1.0000
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 3.3599e-06 - accuracy: 1.0000
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1155e-07 - accuracy: 1.0000


1.0

In [30]:
acc_deg=[]
path='data.csv'
#f = open(path, 'w') # open the file in the write mode
# create the csv writer

for i in range(1,256):
    
    model=define_model()
    (x_train,y_train) = data_generation(1000,3,i,1)
    learn(model,x_train,y_train,10,0)
    learn(model,x_train,y_train,10,0)
    accuracy = learn(model,x_train,y_train,10,0)[1].history['accuracy'][-1]
    poldeg = PolyDegOfRule(i)
    row = (accuracy, poldeg)   
    # write a row to the csv file
    with open(path,'a') as file:
        writer = csv.writer(file)
        writer.writerow(row)
    #acc_deg.append((accuracy,poldeg))
    
# close the file
#f.close()
#acc_deg

GeneratorsNeeded: Cannot initialize from 'dict' without generators

In [ ]:
for i in range(170,180):
    print(PolyDegOfRule(i),": ",ACAP(i),"== ",ACAP(i).subs('x1','x').subs('x2','x').subs('x3','x'))

In [7]:
#print(x_train[0:3])
print(y_train[20:40])
#print(model(x_train)[20:40].numpy())
np.around(tf.keras.activations.sigmoid(model(x_train)[20:40]))
#print(sum(np.around(tf.keras.activations.sigmoid(model.get_weights()[0]))[:],["x1","+ x2","+ x3"][:]))
weights=np.around(tf.keras.activations.sigmoid(model.get_weights()[0]))

NameError: name 'y_train' is not defined